In [1]:
from dapnn.imports import *
from dapnn.data_processing import *
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import torch

import warnings
warnings.filterwarnings(action='once')

In [2]:
# Set the random seed for reproducible results
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed_everything()

In [3]:
log_path='data/csv/PDC2020_training/'
log = import_log(log_path)

num_traces = len(log['trace_id'].unique())
ratio = 0.8
traces_for_normal_model = int(num_traces * ratio)

log = log[log['trace_id'].str.split(' ').str[1].astype(int) > traces_for_normal_model]
log.head(5)

['data/csv/PDC2020_training/pdc_2020_0000000.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0000001.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0000010.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0000011.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0000100.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0000101.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0000110.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0000111.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0001000.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0001001.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0001010.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0001011.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0001100.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0001101.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0001110.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0001111.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0010000.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0010001.csv.gz', 'data/csv/PDC2020_training/pdc_2020_0010010.c

,activity,trace_id,event_id
trace_id,,,
trace 153601,###start###,trace 153601,0
trace 153601,t26,trace 153601,1
trace 153601,t35,trace 153601,2
trace 153601,t44,trace 153601,3
trace 153601,t34,trace 153601,4


# Process Discovery Using The IM

In [4]:
import pandas as pd
import pm4py

In [5]:
log = pm4py.format_dataframe(log, case_id='trace_id', activity_key='activity', timestamp_key='event_id')
log = pm4py.convert_to_event_log(log)
model_str = 'heuristic' # Change

/lv_local/home/romh/PROGS/anaconda3/envs/dapnn/lib/python3.9/site-packages/pm4py/objects/log/util/dataframe_utils.py:180: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/lv_local/home/romh/PROGS/anaconda3/envs/dapnn/lib/python3.9/site-packages/pm4py/objects/log/util/dataframe_utils.py:180: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/lv_local/home/romh/PROGS/anaconda3/envs/dapnn/lib/python3.9/site-packages/pm4py/objects/log/util/dataframe_utils.py:180: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify 

In [13]:
model_str = 'ILP'

In [6]:
def get_process_model(log, model_str):
    if model_str == 'IM':
        net, im, fm = pm4py.discover_petri_net_inductive(log)
    elif model_str == 'alpha':
        net, im, fm = pm4py.discover_petri_net_alpha(log)
    elif model_str == 'heuristic':
        net, im, fm = pm4py.discover_petri_net_heuristics(log)
    return net, im, fm

In [14]:
net, im, fm = get_process_model(log, model_str)

/lv_local/home/romh/PROGS/anaconda3/envs/dapnn/lib/python3.9/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


discovering Petri net using ILP miner, completed causal relations ::   0%|          | 0/108 [00:00<?, ?it/s]

# Test Set

In [8]:
test_path = 'data/csv/PDC2020_ground_truth/'
test_log = import_log(test_path)
test_log.head(5)

['data/csv/PDC2020_ground_truth/pdc_2020_0000000.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0000001.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0000010.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0000011.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0000100.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0000101.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0000110.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0000111.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0001000.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0001001.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0001010.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0001011.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0001100.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0001101.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0001110.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0001111.csv.gz', 'data/csv/PDC2020_ground_truth/pdc_2020_0010000.csv.gz', 'data/csv/PDC2020_ground_truth

,activity,trace_id,case:pdc:costs,normal,event_id
trace_id,,,,,
trace 1,###start###,trace 1,2.0,False,0
trace 1,t11,trace 1,2.0,False,1
trace 1,t21,trace 1,2.0,False,2
trace 1,t26,trace 1,2.0,False,3
trace 1,t35,trace 1,2.0,False,4


In [9]:
df = test_log.copy()
df = df[['trace_id', 'normal']]
df = df.drop_duplicates()

In [10]:
test_log = pm4py.format_dataframe(test_log, case_id='trace_id', activity_key='activity', timestamp_key='event_id')
test_log = pm4py.convert_to_event_log(test_log)

/lv_local/home/romh/PROGS/anaconda3/envs/dapnn/lib/python3.9/site-packages/pm4py/objects/log/util/dataframe_utils.py:180: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/lv_local/home/romh/PROGS/anaconda3/envs/dapnn/lib/python3.9/site-packages/pm4py/objects/log/util/dataframe_utils.py:180: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/lv_local/home/romh/PROGS/anaconda3/envs/dapnn/lib/python3.9/site-packages/pm4py/objects/log/util/dataframe_utils.py:180: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify 

In [15]:
def replay_log(log, net, im, fm ,df, model_str):
    replayed_log = pm4py.conformance_diagnostics_token_based_replay(log, net, im, fm)
    predicted = dict()
    for i, trace in enumerate(replayed_log):
        predicted[f'trace {i+1}'] = not trace['trace_is_fit'] # the trace is anomalous if it is not fit
    predicted = pd.DataFrame.from_dict(predicted, orient='index', columns=['predicted'])
    predicted.index.name = 'trace_id'
    predicted = predicted.join(df, on='trace_id')[['predicted', 'normal']]
    predicted['normal'].replace({True: False, False: True}, inplace=True)
    predicted.columns = ['y_pred', 'y_true']
    predicted.to_csv('save_model/' + model_str + '_y_pred_y_true.csv', index=False)
    return predicted

In [16]:
replay_log(test_log, net, im, fm, df, model_str)

replaying log with TBR, completed variants ::   0%|          | 0/78736 [00:00<?, ?it/s]

,y_pred,y_true
trace_id,,
trace 1,False,True
trace 2,False,False
trace 3,False,True
trace 4,False,True
trace 5,False,False
...,...,...
trace 191996,False,False
trace 191997,False,False
trace 191998,False,True
